In [1]:
%pip install wandelbots
%pip install python-dotenv


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from wandelbots import Instance, MotionGroup, Planner
from wandelbots.types import Pose

load_dotenv()

In [3]:
os.environ["WANDELAPI_BASE_URL"] = "http://172.30.0.199"
# os.environ["NOVA_USERNAME"] = "wb"
# os.environ["NOVA_PASSWORD"] = "your_password"
os.environ["CELL_ID"] = "cell"
os.environ["MOTION_GROUP"] = "0@myrobot"
os.environ["TCP"] = "Flange"

## 🤖 Show Robot-Pad

In [4]:
from IPython.display import IFrame

IFrame(src=os.getenv("WANDELAPI_BASE_URL") + "/cell/robot-pad", width=700, height=600)

## ⚙️ Configure Robot

In [5]:
my_instance = Instance(
    url=os.getenv("WANDELAPI_BASE_URL")
    # user=os.getenv("NOVA_USERNAME"),
    # password=os.getenv("NOVA_PASSWORD"),
)

my_robot = MotionGroup(
    instance=my_instance,
    cell=os.getenv("CELL_ID"),
    motion_group=os.getenv("MOTION_GROUP"),
    default_tcp=os.getenv("TCP"),
)

## 🔩 Get Robot Props & State

In [6]:
print(my_robot.tcps())
print([round(j, 2) for j in my_robot.current_joints()])
print(my_robot.current_tcp_pose())

['Flange']
[1.17, -1.66, 1.4, -1.57, -1.57, 1.17]
Position: [  -91,  -662,   851]
Rotation: [ 2.14,  2.14, -0.36]


## 📝 Perform Pose Transforms

In [7]:
current_pose = my_robot.current_tcp_pose()
translation = Pose.from_list([0, 0, -400, 0, 0, 0])
target_pose = translation * current_pose
print(f"Current\n{current_pose}\n\nTarget\n{target_pose}")

Current
Position: [  -91,  -662,   851]
Rotation: [ 2.14,  2.14, -0.36]

Target
Position: [  -91,  -662,   451]
Rotation: [ 2.14,  2.14, -0.36]


## 🎢 Plan a Motion

In [21]:
HOME = my_robot.current_joints()

planner = Planner(my_robot)
trajectory = [
    planner.jptp(target=HOME),
    planner.io(key="tool_out[0]", value=True),
    planner.line(target=target_pose),
    planner.io(key="tool_out[1]", value=False),
    planner.jptp(target=HOME),
]
plan_result, io_actions = planner.plan(start_joints=my_robot.current_joints(), trajectory=trajectory)

print("motion id:", plan_result.motion)

motion id: 85e25535-5649-4e54-ad55-25c28d726ab2


## 🏃🏽‍♀️ Execute Motion

In [22]:
io_names = [action.io.key for action in io_actions]
ios = my_robot.get_ios(io_names)
print(f"{ios[0]}\n{ios[1]}")

tool_out[0]: False
tool_out[1]: True


In [23]:
result = await my_robot.execute_motion_with_ios_async(motion=plan_result.motion, speed=25, io_values=io_actions)

Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response
Got Response


In [24]:
ios = my_robot.get_ios(io_names)
print(f"{ios[0]}\n{ios[1]}")

tool_out[0]: True
tool_out[1]: False
